convert tmx to json of structure:

!!!mind that fn for learners comes from filesource and for prof - from prop!!!
[
  { "fn" : "EN_1_10.txt", 
    "en" : "string of all tagged sents", # if it is a string I won't be able to pull out aligned sents! I nedd an indexable list here!
    "ru" : "все предложения из этого текста в одну строку"
  } ,
  { "fn" : "EN_1_100.txt",
    "en" : ["", ""]
    "ru" : ["", ""]
  } 
]


In [1]:
import sys,codecs,os
from xml.dom import minidom
import json

In [ ]:
# json how-to
arr  = []

    for pair in sorted(dict, key=dict.get, reverse=True):
    #     print type(pair[0])
    #     print "from:", pair[0], "to:", pair[1], "cos:", dict[pair]
        arr.append({ "from": pair[0].decode('utf-8'), "to": pair[1].decode('utf-8'), "cos": dict[pair]})

    with io.open('/home/masha/voronovo/all_jsons/' + m.split('.')[0] + '.json', 'w', encoding='utf-8') as json_file:
        data = json.dumps(arr, ensure_ascii=False)
        json_file.write(data)

In [2]:
arg = '/home/masha/birmingham/data/EM_my_uniq200.tmx' #sys.argv[1]

In [3]:
doc = minidom.parse(arg)
tus = doc.getElementsByTagName("tu") #возвращает список tu (элемента, содержащего tuvs)

In [4]:
# loop over all tuvs in tus to create a list of keys (=fn) in the statistics and restore_text dics; these dics have lists of values for each key
fns = []
for tu in tus:
    tuvs = tu.getElementsByTagName("tuv")
    for tuv in tuvs:
        fn = tuv.getAttribute('filesource')
        fn = fn.strip()
        if fn not in fns:
            fns.append(fn)
restore_texts = {fn : [] for fn in fns}
statistics = {fn :{} for fn in fns}


In [5]:
# build the restore_texts dic and collect all values (=text segments)
print len(tus)
fn_in_restore = 0
for tu in tus:
    tuvs = tu.getElementsByTagName("tuv")
    for tuv in tuvs:
        fn = tuv.getAttribute('filesource')

        fn = fn.strip()
        seg_el = tuv.getElementsByTagName("seg")[0]

        seg_text = seg_el.childNodes[-1].data
        seg_text = seg_text.strip()
        
        restore_texts[fn].append(seg_text) # there are 19524 segs in my_uniq200.tmx
    #print len(restore_texts[fn])

9762


In [6]:
# produce number of sentences for each file, which can be accessed as restore_segs[fn]

for fn, segs in restore_texts.items():
    num_SENTs = 0
    for seg in segs: #this replace obviously did not work
        SENTs = seg.count('_SENT_')
        num_SENTs += SENTs
    print fn, '\t', num_SENTs
    statistics[fn]['SENTs'] = num_SENTs

EN_1_263.head.txt 	10
RU_1_67_1.head.txt 	576
EN_1_48.head.txt 	23
EN_1_145.head.txt 	19
EN_1_50.head.txt 	33
EN_1_296.head.txt 	26
EN_4_40.head.txt 	439
EN_1_266.head.txt 	15
RU_1_289_1.head.txt 	22
RU_1_116_1.head.txt 	13
EN_1_19.head.txt 	26
RU_1_243_1.head.txt 	179
RU_1_180_1.head.txt 	65
RU_1_139_1.head.txt 	408
RU_1_292_1.head.txt 	16
EN_1_144.head.txt 	7
RU_1_247_1.head.txt 	46
EN_3_2.head.txt 	10
RU_1_207_1.head.txt 	28
EN_1_126.head.txt 	14
EN_1_3.head.txt 	40
EN_1_52.head.txt 	19
EN_1_100.head.txt 	21
RU_3_4_1.head.txt 	24
RU_1_92_1.head.txt 	118
RU_1_170_1.head.txt 	29
EN_1_312.head.txt 	51
RU_1_80_1.head.txt 	16
EN_5_6.head.txt 	9
RU_4_39_1.head.txt 	51
RU_3_5_1.head.txt 	20
RU_6_2_1.head.txt 	14
RU_1_203_1.head.txt 	25
RU_1_303_1.head.txt 	22
EN_1_2.head.txt 	39
RU_1_245_1.head.txt 	20
EN_1_212.head.txt 	24
RU_1_264_1.head.txt 	19
RU_4_40_1.head.txt 	438
RU_1_237_1.head.txt 	18
EN_1_315.head.txt 	38
RU_3_13_1.head.txt 	8
EN_1_122.head.txt 	16
RU_1_68_1.head.txt 	99
RU_1_53

count freqs of tagged items - CONNs and EMs, 

calculate their normalized freqs using stats from statistics and append

In [7]:
queries = set([item.strip() for item in codecs.open(arg2, 'r', 'utf-8').readlines()])
print 'Number of search items: ', len(queries)

total_hits = {item : [] for item in queries}
nfreqs = {item : [] for item in queries}
for item in queries:
#     count_item = 0 # counter for freq of all items in this fn
    for fn, segs in restore_texts.items(): # segs is a list
        string = " ".join(seg for seg in segs) #convert list of segs to a string to avoid yet another loop
        '''
        Despite "The method count() returns count of how many times obj occurs in list," 
        count() does not work with seg list!
        !!!why does it skip the first list element? (EAEM_becertain) Works if the list starts with a blank line
        '''
        hits = string.count(item) # why did I have .find() here?!! 
#         count_item +=hits
        total_hits[item].append(hits)
        nfreqs[item].append(hits/statistics[fn]['SENTs']*100) # list of normalized freq for each text
        if hits > 0:
            print item.encode('utf-8'), sum(total_hits[item])
    #print fn, '\t', count_item

Number of search items:  160
EAEM_likely 1
EAEM_likely 8
EAEM_likely 9
EAEM_likely 10
EAEM_likely 11
EAEM_likely 12
EAEM_likely 15
EAEM_likely 18
EAEM_likely 19
EAEM_likely 22
EAEM_likely 23
EAEM_likely 25
EAEM_likely 26
EAEM_likely 27
EAEM_likely 28
EAEM_likely 29
EAEM_likely 30
EAEM_likely 32
EAEM_likely 33
EAEM_likely 34
EAEM_likely 35
EAEM_likely 36
EAEM_likely 39
EAEM_likely 40
EAEM_likely 42
EAEM_likely 43
EAEM_likely 44
EAEM_likely 45
EAEM_likely 46
EAEM_likely 47
EAEM_likely 48
EAEM_likely 54
EAEM_likely 55
EVEM_предполагаю 1
EVEM_предполагаю 2
EVEM_предполагаю 3
EAEM_bepossible 1
EAEM_bepossible 2
EVEM_несчитаю 1
EVEM_несчитаю 2
EVEM_iassume 1
EVEM_seemthat 1
EAEM_itisclear 2
EAEM_itisclear 3
EAEM_itisclear 4
EAEM_itisclear 5
EVEM_мнеНЕкажетсячто 1
OWEM_certainly 3
OWEM_certainly 4
OWEM_certainly 6
OWEM_certainly 7
OWEM_certainly 8
OWEM_certainly 9
OWEM_certainly 10
OWEM_certainly 11
OWEM_certainly 12
OWEM_certainly 13
OWEM_certainly 14
OWEM_certainly 15
OWEM_certainly 16
OWEM

EVEM_LINKseems 10
EVEM_LINKseems 11
EVEM_LINKseems 13
EVEM_LINKseems 14
EVEM_LINKseems 15
EVEM_LINKseems 16
EVEM_LINKseems 17
EVEM_LINKseems 18
EVEM_LINKseems 22
MWEM_ofcourse 7
MWEM_ofcourse 8
MWEM_ofcourse 9
MWEM_ofcourse 10
MWEM_ofcourse 11
MWEM_ofcourse 12
MWEM_ofcourse 15
MWEM_ofcourse 16
MWEM_ofcourse 18
MWEM_ofcourse 19
MWEM_ofcourse 22
MWEM_ofcourse 23
MWEM_ofcourse 24
MWEM_ofcourse 25
MWEM_ofcourse 26
MWEM_ofcourse 27
MWEM_ofcourse 28
MWEM_ofcourse 29
MWEM_ofcourse 30
MWEM_ofcourse 31
MWEM_ofcourse 33
MWEM_ofcourse 34
MWEM_ofcourse 35
MWEM_ofcourse 37
MWEM_ofcourse 47
MWEM_ofcourse 48
OWEM_вроде 1
OWEM_вроде 2
OWEM_вроде 3
MWEM_безсомнения 1
MWEM_безсомнения 2
MWEM_безсомнения 3
MWEM_безсомнения 4
MWEM_безсомнения 5
OWEM_вероятно 5
OWEM_вероятно 7
OWEM_вероятно 9
OWEM_вероятно 13
OWEM_вероятно 14
OWEM_вероятно 20
OWEM_вероятно 21
OWEM_вероятно 24
OWEM_вероятно 25
OWEM_вероятно 26
OWEM_вероятно 27
OWEM_вероятно 38
OWEM_вероятно 39
OWEM_вероятно 45
OWEM_вероятно 46
OWEM_вероятно

In [8]:
for item in queries: 
    print item.split('_')[1].encode('utf-8'), sum(total_hits[item]) # lost tags but good for sorting
    

likely 55
предполагаю 3
bepossible 2
несчитаю 2
iassume 1
seemthat 1
tomymind 0
надополагать 0
itisclear 5
можнонесомневаться 0
goeswithoutsaying 0
мнеНЕкажетсячто 1
certainly 29
кажетсячто 10
unlikely 3
несомневаюсь 0
неужели 3
надодумать 0
seemtome 2
самособойразумеется 1
мнеНЕкажется 3
кажется 15
каквидно 0
считаю 8
думаю 17
нетсомнениячто 0
ibelievethat 3
какпредставляется 0
ibelieve 5
думается 0
decidedly 1
самособой 1
какпредполагается 0
наверно 7
допускаю 0
какпредполагается 0
havenodoubt 0
benotlikely 0
becertainto 0
высказалипредположение 0
beobvious 0
sure 1
obviously 8
definitely 4
уверен 4
maybe 20
такли 6
маловероятный 1
возможно 92
LINKseem 27
arguably 4
asforme 1
полагаю 1
looklike 0
полагаюнеобходимым 0
dontbelieve 1
ifeel 0
неубежден 0
theredoubt 2
astome 0
inmyview 0
естьпредположениечто 0
havedoubt 0
sureenough 1
benotcertain 0
можноутверждать 4
benotlikelyto 0
bestofmyknowledge 0
havefeeling 0
nodoubt 2
seemto 18
какожидается 1
forsure 0
утверждаю 1
смоейточкизрения

In [9]:
print 'lemma\t total\t av_nfreq*100\n'
for item in queries:
    total = sum(total_hits[item])
    av_nfreq = round(np.mean(nfreqs[item]),4)*2 # mean normalized frequency of the item over all texts (*2 accounts for bi-lingual char of the format)

    print item.split('_')[1].encode('utf-8')+'\t', total,'\t', av_nfreq

lemma	 total	 av_nfreq*100

likely	55 	0.6282
предполагаю	3 	0.0198
bepossible	2 	0.0572
несчитаю	2 	0.0162
iassume	1 	0.0012
seemthat	1 	0.0006
tomymind	0 	0.0
надополагать	0 	0.0
itisclear	5 	0.0512
можнонесомневаться	0 	0.0
goeswithoutsaying	0 	0.0
мнеНЕкажетсячто	1 	0.0006
certainly	29 	0.296
кажетсячто	10 	0.2036
unlikely	3 	0.0104
несомневаюсь	0 	0.0
неужели	3 	0.0118
надодумать	0 	0.0
seemtome	2 	0.0498
самособойразумеется	1 	0.0012
мнеНЕкажется	3 	0.0222
кажется	15 	0.0812
каквидно	0 	0.0
считаю	8 	0.1238
думаю	17 	0.1398
нетсомнениячто	0 	0.0
ibelievethat	3 	0.0494
какпредставляется	0 	0.0
ibelieve	5 	0.092
думается	0 	0.0
decidedly	1 	0.0218
самособой	1 	0.0012
какпредполагается	0 	0.0
наверно	7 	0.0278
допускаю	0 	0.0
какпредполагается	0 	0.0
havenodoubt	0 	0.0
benotlikely	0 	0.0
becertainto	0 	0.0
высказалипредположение	0 	0.0
beobvious	0 	0.0
sure	1 	0.0108
obviously	8 	0.0826
definitely	4 	0.0244
уверен	4 	0.0444
maybe	20 	0.186
такли	6 	0.0054
маловероятный	1 	0.0008
воз

In [10]:
with codecs.open('/home/masha/birmingham/stats/test_EM-by-item_uniq200.tsv', 'w') as outfile:
    writer = csv.writer(outfile, delimiter='\t')
    writer.writerow(['item'] + ['all_hits'] + ['av_nfreq']) #table header
    for item in queries:
        writer.writerow([item.split('_')[1].encode('utf-8')] + [sum(total_hits[item])] + [round(np.mean(nfreqs[item]),4)*2])
    

creat a dic with items as keys and respective lists of normalized frequencies for each text as values

In [11]:
# items_stats = {item : {} for item in queries} # the dic structure: {item:{fn:[count_item, SENTs, nfreq]}}
# for item in queries:
#     count_item = 0 # counter for all fns
#     nfreq = [] # list of nfreqs for this item, its length = num of files
#     for fn, segs in restore_texts.items():
#         hits = 0 # counter for all occurences in this fn
#         for seg in segs:
#             hits += seg.count(item)# hits in each seg
#         count_item += hits #all hits in each fn
#         nfreq = count_item/statistics[fn]['SENTs']*100
#         items_stats[item] = {fn : [count_item]}
#         items_stats[item][fn].append(statistics[fn]['SENTs'])
#         items_stats[item][fn].append(count_item/statistics[fn]['SENTs']*100)
# for fn, ls in items_stats[item].items():
#     print fn, items_stats[item][fn]
#     #print items_stats[item][fn]
# #         items_stats[item].append(nfreq)
# #         av_nfreq = np.mean(items_stats[item])
# #         if av_nfreq > 0:
        
# #             print item.encode('utf-8'), av_nfreq

